In [1]:
import pandas as pd

In [2]:
test_data_df = pd.read_fwf('test.txt',delimiter="\n")
test_data_df.columns = ["Text"]
test_data_df.head()

,Text
0,"""I love it"""
1,"""I am so happy, this is great"""
2,"""I am so angry, this sucks"""
3,"""I am so happy, this is great"""
4,"""I am so happy, this is great"""


In [3]:
train_data_neg_df = pd.read_fwf('neg_tweets.txt',delimiter="\n")
train_data_neg_df.insert(0, 'Sentiment', 0)
train_data_neg_df.columns = ["Sentiment","Text"]
train_data_neg_df.head()

,Sentiment,Text
0,0,""" cavs lost, and I got this sinking feeling we..."
1,0,""" the closest BGT tour is Cardiff or London da..."
2,0,""" Why do other pet care people try to run othe..."
3,0,"""-gasps- dananananaykroyd touring aussie .... ..."
4,0,"""....If i am going warsal that means no church..."


In [4]:
train_data_pos_df = pd.read_fwf('pos_tweets.txt',delimiter="\n")
train_data_pos_df.insert(0, 'Sentiment', 1)
train_data_pos_df.columns = ["Sentiment","Text"]
train_data_pos_df.head()

,Sentiment,Text
0,1,""" just watched the movie Wanted... it was pret..."
1,1,""" now I'm happy """
2,1,"""--plotting like i'm mike..'game plan:pass the..."
3,1,"""@ mcdonalds with my litto sis aka cuzin lol c..."
4,1,"""@ PBnJen : Thanks for the great tour and maki..."


In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection  import train_test_split

In [ ]:
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)
    stems = [PorterStemmer().stem(item) for item in tokens]
    return(tokens)

In [ ]:
vectorizer = CountVectorizer(analyzer = 'word',tokenizer = tokenize,lowercase = True,stop_words = 'english',max_features = 80)

In [ ]:
train_data_df = pd.concat([train_data_pos_df,train_data_neg_df])
features = vectorizer.fit_transform(train_data_df.Text.tolist()+test_data_df.Text.tolist())
features_nd = features.toarray()

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(features_nd[0:len(train_data_df)], train_data_df.Sentiment,test_size=0.1, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver='lbfgs')

In [ ]:
log_model = log_model.fit(X=features_nd[0:len(train_data_df)], y=train_data_df.Sentiment)

In [ ]:
y_pred = log_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       142
           1       0.86      0.54      0.67        59

   micro avg       0.84      0.84      0.84       201
   macro avg       0.85      0.75      0.78       201
weighted avg       0.84      0.84      0.83       201



In [ ]:
a = input("Type a sentence and press enter")

In [ ]:
test_data_df2 = pd.DataFrame({'Text':[a]}) 
test_data_df2.head()

In [ ]:
log_model = LogisticRegression(solver='lbfgs')
log_model = log_model.fit(X=features_nd[0:len(train_data_df)], y=train_data_df.Sentiment)
test_pred = log_model.predict(features_nd[len(train_data_df):])
print(test_data_df2.Text[0], test_pred[0])

In [ ]:
import praw

reddit = praw.Reddit(client_id='8G3apa5U51Rw2g',
                     client_secret='ACPdbSJSDLV8U3EAjla2MhOj1zg',
                     user_agent='my user agent')

print(reddit.read_only) 

In [ ]:
df = pd.DataFrame(columns=['Text'])
for submission in reddit.subreddit('upliftingnews').top('year',limit=100):
    df = df.append({'Text': submission.title}, ignore_index=True)
#log_model = LogisticRegression(solver='lbfgs')
#log_model = log_model.fit(X=features_nd[0:len(train_data_df)], y=train_data_df.Sentiment)
test_pred = log_model.predict(features_nd[len(df):])
for i in range(len(df)):
    print(df.Text[i], test_pred[i])

In [ ]:
log_model = LogisticRegression(solver='lbfgs')
log_model = log_model.fit(X=features_nd[0:len(train_data_df)], y=train_data_df.Sentiment)
test_pred = log_model.predict(features_nd[len(train_data_df):])
import random
spl = random.sample(range(len(test_pred)), 6)
for text, sentiment in zip(test_data_df2.Text[spl], test_pred[spl]):
    print (sentiment, text)

In [ ]:
#for i in range(0, 5):
#    print(test_data_df.Text[i], test_pred[i])